## Open notebook in:
| Colab                               Gradient                                                                                                                                         |
|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nicolepcx/Transformers-in-Action/blob/main/CH08/ch08_ray_tune.ipynb)                                              

# Imports


# About this notebook

This notebook is inspired by two examples from Ray:

* [Example one](https://docs.ray.io/en/latest/train/getting-started-transformers.html)
* [Example two](https://docs.ray.io/en/latest/train/examples/transformers/huggingface_text_classification.html)

You will use one dataset from the [superclue dataset](https://huggingface.co/datasets/super_glue).

This notebook show the following steps:
1. Set up Ray
2. Load the dataset and process with Ray Data
3. Run the training with Ray Train



In [ ]:
!pip install evaluate -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
!pip install ray -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 34.7 MB/s eta 0:00:00


In [ ]:
!pip install datasets -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12


## Set up Ray

Use `ray.init()` to initialize a local cluster. By default, this cluster contains only the machine you are running this notebook on.

In [ ]:
from pprint import pprint
import ray

ray.init(num_cpus=1, num_gpus=1, include_dashboard=False)

2025-07-14 05:29:03,365	INFO worker.py:1917 -- Started a local Ray instance.


Python version:,3.11.13
Ray version:,2.47.1


Check the resources our cluster is composed of. If you are running this notebook on your local machine or Google Colab, you should see the number of CPU cores and GPUs available on the your machine.

In [ ]:
pprint(ray.cluster_resources())

{'CPU': 1.0,
 'GPU': 1.0,
 'accelerator_type:A100': 1.0,
 'memory': 62497129677.0,
 'node:172.28.0.12': 1.0,
 'node:__internal_head__': 1.0,
 'object_store_memory': 26784484147.0}


# Imports

In [ ]:
import os
import torch
import numpy as np

import ray.data
from datasets import load_dataset
from transformers import AutoTokenizer
import wandb

import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from huggingface_hub import notebook_login

import ray.train
from ray.train import RunConfig, ScalingConfig, CheckpointConfig, Checkpoint
from ray.train.huggingface.transformers import prepare_trainer, RayTrainReportCallback
from ray.train.torch import TorchTrainer

from ray import tune
from ray.tune import Tuner
from ray.tune.schedulers.async_hyperband import ASHAScheduler


In [ ]:
use_gpu = True  # set this to False to run on CPUs
num_workers = 1  # set this to number of GPUs or CPUs you want to use

## Log into weights & biases

In [ ]:
wandb.login(key='your_key')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nicolepcx to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Set the task for SuperGlue

In [ ]:
task = "cb"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16


## Load the dataset

In [ ]:
hf_dataset = load_dataset("super_glue", task)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/58.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/63.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/56 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
def is_valid(example):
    return isinstance(example.get("premise"), str) and isinstance(example.get("hypothesis"), str)

hf_dataset["train"] = hf_dataset["train"].filter(is_valid)
hf_dataset["validation"] = hf_dataset["validation"].filter(is_valid)
hf_dataset["test"] = hf_dataset["test"].filter(is_valid)


Filter:   0%|          | 0/250 [00:00<?, ? examples/s]

Filter:   0%|          | 0/56 [00:00<?, ? examples/s]

Filter:   0%|          | 0/250 [00:00<?, ? examples/s]

## Preprocessing the data with Ray Data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
ray_datasets = {
    "train": ray.data.from_items(hf_dataset["train"].to_list()),
    "validation": ray.data.from_items(hf_dataset["validation"].to_list()),
    "test": ray.data.from_items(hf_dataset["test"].to_list()),
}
ray_datasets

{'train': MaterializedDataset(
    num_blocks=200,
    num_rows=250,
    schema={premise: string, hypothesis: string, idx: int64, label: int64}
 ),
 'validation': MaterializedDataset(
    num_blocks=56,
    num_rows=56,
    schema={premise: string, hypothesis: string, idx: int64, label: int64}
 ),
 'test': MaterializedDataset(
    num_blocks=200,
    num_rows=250,
    schema={premise: string, hypothesis: string, idx: int64, label: int64}
 )}

## Preprocesses the samples

In [ ]:
def tokenize_fn(batch):
    premises = batch["premise"]
    hypotheses = batch["hypothesis"]
    labels = batch["label"]

    # Cast to list to ensure compatibility (HF tokenizer can handle np.array, but torch.tensor prefers lists here)
    premises = premises.tolist() if isinstance(premises, np.ndarray) else premises
    hypotheses = hypotheses.tolist() if isinstance(hypotheses, np.ndarray) else hypotheses
    labels = labels.tolist() if isinstance(labels, np.ndarray) else labels

    # Tokenize
    tokenized = tokenizer(
        premises,
        hypotheses,
        truncation=True,
        padding="longest",
        return_tensors="pt"
    )

    # Add labels
    tokenized["labels"] = torch.tensor(labels, dtype=torch.long)
    tokenized = {k: v.to("cuda") for k, v in tokenized.items()}
    return tokenized


## Fine-tuning the model with Ray Train

Now that the data is ready, download the pretrained model and fine-tune it.

You need to define the training logic as a function (`train_func`). You pass this [training function] to the {class}`~ray.train.torch.TorchTrainer` and with that on every Ray worker.


In [ ]:
num_labels = 3
metric_name = ("accuracy")

model_name = model_checkpoint.split("/")[-1]
validation_key = ("validation")

name = f"{model_name}-finetuned-{task}"

# Calculate the maximum steps per epoch based on the number of rows in the training dataset.
max_steps_per_epoch = ray_datasets["train"].count() // (batch_size * num_workers)


def train_func(config):
    print(f"CUDA available: {torch.cuda.is_available()}")

    # Load metric
    metric = evaluate.load("super_glue", config_name=task)

    # Tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint, num_labels=num_labels
    )

    # Load Ray datasets
    train = ray.train.get_dataset_shard("train")
    eval = ray.train.get_dataset_shard("eval")

    train_iterable = train.iter_torch_batches(
        batch_size=batch_size, collate_fn=tokenize_fn
    )
    eval_iterable = eval.iter_torch_batches(
        batch_size=batch_size, collate_fn=tokenize_fn
    )

    # Training arguments
    args = TrainingArguments(
        name,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        per_device_train_batch_size=config.get("batch_size", 64),
        per_device_eval_batch_size=config.get("batch_size", 64),
        learning_rate=config.get("learning_rate", 2e-5),
        num_train_epochs=config.get("epochs", 6),
        weight_decay=config.get("weight_decay", 0.001),
        max_steps=max_steps_per_epoch * config.get("epochs", 6),
        disable_tqdm=False,
        no_cuda=not torch.cuda.is_available(),
        report_to="wandb",
        run_name="superglue_cb"
    )

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return metric.compute(predictions=predictions, references=labels)

    # Trainer setup
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_iterable,
        eval_dataset=eval_iterable,
        compute_metrics=compute_metrics,
    )
    trainer.add_callback(RayTrainReportCallback())
    trainer = prepare_trainer(trainer)

    print("Beginning training...")
    trainer.train()

    # Final evaluation
    eval_metrics = trainer.evaluate()
    print("Final evaluation:", eval_metrics)

    # Save HF-compatible model directory
    output_dir = os.path.join(ray.train.get_context().get_trial_dir(), "hf_model")
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Report metrics + HF-style checkpoint to Ray Tune
    ray.train.report(
        {
            "eval_loss": eval_metrics.get("eval_loss"),
            "eval_accuracy": eval_metrics.get("eval_accuracy"),
        },
        checkpoint=Checkpoint.from_directory(output_dir)
    )


## Setup TorchTrainer


Having finalized your train_func, it's time to create an instance of the `{class}~ray.train.torch.TorchTrainer`. In addition to invoking the function, configure the `scaling_config` to manage the number of workers and resources allocated, as well as specify the `datasets` for training and evaluation purposes.

In [ ]:
trainer = TorchTrainer(
    train_func,
    scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
    datasets={
        "train": ray_datasets["train"],
        "eval": ray_datasets["validation"],
    },
    run_config=RunConfig(
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="eval_loss",
            checkpoint_score_order="min",
        ),
    ),
)

## Tune hyperparameters with Ray Tune

To adjust the model's hyperparameters, insert your TorchTrainer into a Tuner and specify the search space. I have added some example hyperparameters.

Moreover, you can leverage the sophisticated search algorithms and schedulers provided by `Ray Tune`. In this instance, an `ASHAScheduler` is utilized to aggressively halt trials that are not performing well.

In [ ]:
tuner = Tuner(
    trainer,
    param_space={
        "train_loop_config": {
            "learning_rate": tune.grid_search([2e-5, 2e-4, 2e-3, 2e-2]),
            "epochs": tune.choice([2, 4, 6, 8]),  # Experiment with these epoch values
            "batch_size": tune.choice([16, 32, 64, 128]),  # Experiment with these batch sizes
            "weight_decay": tune.grid_search([0.0, 0.01, 0.1, 0.001])  # Experiment with these weight decay values
        }
    },
    tune_config=tune.TuneConfig(
        metric="eval_loss",
        mode="min",
        num_samples=1,
        scheduler=ASHAScheduler(
            max_t=max([2, 4, 6, 8]),  # Set to the maximum value of epochs being considered
            grace_period=1,  # Minimum number of epochs to evaluate before considering stopping
            reduction_factor=2,  # Reduction factor for pruning
        ),
    ),
    run_config=RunConfig(
        name="tune_transformers",
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="eval_loss",
            checkpoint_score_order="min",
        ),
    ),
)


2025-07-14 05:29:32,081	INFO tuner_internal.py:427 -- A `RunConfig` was passed to both the `Tuner` and the `TorchTrainer`. The run config passed to the `Tuner` is the one that will be used.


## Tune the model

In [ ]:
tune_results = tuner.fit()

2025-07-14 05:29:32,126	INFO tensorboardx.py:193 -- pip install "ray[tune]" to see TensorBoard files.
2025-07-14 05:29:32,127	WARNING callback.py:136 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


+----------------------------------------------------------+
| Configuration for experiment     tune_transformers       |
+----------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator   |
| Scheduler                        AsyncHyperBandScheduler |
| Number of trials                 16                      |
+----------------------------------------------------------+

View detailed results here: /root/ray_results/tune_transformers

Trial status: 16 PENDING
Current time: 2025-07-14 05:29:32. Total running time: 0s
Logical resource usage: 0/1 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:A100)
+-----------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                 status       ...oop_config/epochs     ...config/batch_size     ...fig/learning_rate     ...nfig/weight_decay |
+--------------------------------------------------------------------

(TrainTrainable pid=3042) 2025-07-14 05:29:38.070522: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(TrainTrainable pid=3042) 2025-07-14 05:29:38.088623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(TrainTrainable pid=3042) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(TrainTrainable pid=3042) E0000 00:00:1752470978.110287    3042 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(TrainTrainable pid=3042) E0000 00:00:1752470978.116726    3042 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting t


Trial TorchTrainer_853b7_00000 started with configuration:
+-------------------------------------------------+
| Trial TorchTrainer_853b7_00000 config           |
+-------------------------------------------------+
| train_loop_config/batch_size                128 |
| train_loop_config/epochs                      6 |
| train_loop_config/learning_rate           2e-05 |
| train_loop_config/weight_decay                0 |
+-------------------------------------------------+


(RayTrainWorker pid=3139) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=3042) Started distributed worker processes: 
(TorchTrainer pid=3042) - (node_id=dfac881cdf0b95ec3c65ef7c281e0d5ea5b341dd9a6d97ecd807b1a3, ip=172.28.0.12, pid=3139) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=3139) 2025-07-14 05:29:47.943008: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(RayTrainWorker pid=3139) 2025-07-14 05:29:47.959585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(RayTrainWorker pid=3139) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(RayTrainWorker pid=313

...

(pid=10787) - split(1, equal=True) 1: 0.00 row [00:00, ? row/s]

 13%|█▎        | 16/120 [00:01<00:07, 13.45it/s]


(RayTrainWorker pid=10669) {'eval_loss': 23.915176391601562, 'eval_accuracy': 0.4107142857142857, 'eval_f1': 0.1940928270042194, 'eval_runtime': 0.22, 'eval_samples_per_second': 254.49, 'eval_steps_per_second': 9.089, 'epoch': 0.13}

Trial TorchTrainer_853b7_00015 completed after 1 iterations at 2025-07-14 05:40:02. Total running time: 10min 30s
+-------------------------------------------------------------+
| Trial TorchTrainer_853b7_00015 result                       |
+-------------------------------------------------------------+
| checkpoint_dir_name                       checkpoint_000000 |
| time_this_iter_s                                   19.07696 |
| time_total_s                                       19.07696 |
| training_iteration                                        1 |
| epoch                                               0.13333 |
| eval_accuracy                                       0.41071 |
| eval_f1                                             0.19409 |
| eval_loss 

(RayTrainWorker pid=10669) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/tune_transformers/TorchTrainer_853b7_00015_15_batch_size=32,epochs=8,learning_rate=0.0200,weight_decay=0.0010_2025-07-14_05-29-32/checkpoint_000000)
(SplitCoordinator pid=10787) Registered dataset logger for dataset eval_34_0
(SplitCoordinator pid=10787) Starting execution of Dataset eval_34_0. Full logs are in /tmp/ray/session_2025-07-14_05-29-01_733730_2160/logs/ray-data
(SplitCoordinator pid=10787) Execution plan of Dataset eval_34_0: InputDataBuffer[Input] -> OutputSplitter[split(1, equal=True)]
2025-07-14 05:40:10,442	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/tune_transformers' in 7.7316s.



Trial status: 16 TERMINATED
Current time: 2025-07-14 05:40:10. Total running time: 10min 38s
Logical resource usage: 0/1 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Current best trial: 853b7_00009 with eval_loss=0.7809239625930786 and params={'train_loop_config': {'learning_rate': 0.0002, 'epochs': 2, 'batch_size': 16, 'weight_decay': 0.1}}
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                 status         ...oop_config/epochs     ...config/batch_size     ...fig/learning_rate     ...nfig/weight_decay     iter     total time (s)      loss     grad_norm     learning_rate      epoch |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## View Training results

View the results of the tuning run as a dataframe.

In [ ]:
tune_results.get_dataframe().sort_values("eval_loss")

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_accuracy,eval_f1,eval_runtime,eval_samples_per_second,...,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/train_loop_config/learning_rate,config/train_loop_config/epochs,config/train_loop_config/batch_size,config/train_loop_config/weight_decay,logdir
9,NaN,NaN,NaN,NaN,NaN,0.780924,0.678571,NaN,NaN,NaN,...,7513,fe77ef01b3ab,172.28.0.12,31.517041,3,0.00020,2,16,0.100,853b7_00009
12,NaN,NaN,NaN,NaN,NaN,0.820447,0.696429,NaN,NaN,NaN,...,8946,fe77ef01b3ab,172.28.0.12,50.806218,5,0.00002,4,64,0.001,853b7_00012
0,NaN,NaN,NaN,NaN,NaN,0.845492,0.678571,NaN,NaN,NaN,...,3042,fe77ef01b3ab,172.28.0.12,57.555266,7,0.00002,6,128,0.000,853b7_00000
8,NaN,NaN,NaN,NaN,NaN,0.906519,0.535714,NaN,NaN,NaN,...,7012,fe77ef01b3ab,172.28.0.12,28.175880,3,0.00002,2,32,0.100,853b7_00008
4,0.9009,3.373150,6.666667e-07,1.466667,30.0,0.911185,0.589286,0.395522,0.1850,302.643,...,5145,fe77ef01b3ab,172.28.0.12,26.998468,2,0.00002,2,128,0.010,853b7_00004
13,1.0283,1.396966,9.666667e-05,1.266667,32.0,0.953050,0.500000,0.222222,0.1939,288.878,...,9575,fe77ef01b3ab,172.28.0.12,31.527412,2,0.00020,4,16,0.001,853b7_00013
1,0.9918,2.880574,1.500000e-04,0.266667,16.0,1.011027,0.500000,0.222222,0.2131,262.767,...,3817,fe77ef01b3ab,172.28.0.12,20.134871,1,0.00020,4,16,0.000,853b7_00001
5,0.9997,5.086144,1.500000e-04,0.266667,16.0,1.069740,0.500000,0.222222,0.2119,264.221,...,5635,fe77ef01b3ab,172.28.0.12,19.229907,1,0.00020,4,32,0.010,853b7_00005
2,1.5553,14.364967,1.500000e-03,0.266667,16.0,1.420181,0.500000,0.222222,0.2071,270.416,...,4259,fe77ef01b3ab,172.28.0.12,19.510899,1,0.00200,4,16,0.000,853b7_00002
14,2.0457,8.499955,1.750000e-03,0.133333,16.0,1.423099,0.500000,0.222222,0.2260,247.799,...,10093,fe77ef01b3ab,172.28.0.12,20.191509,1,0.00200,8,128,0.001,853b7_00014


## Get best hyperparameters

In [ ]:
best_hyperparameters = tune_results.get_best_result()

In [ ]:
best_trial = tune_results.get_best_result(metric="eval_loss", mode="min")
train_loop_config = best_trial.config['train_loop_config']

## Train the model with best hyperparameters

In [ ]:
# Preparing the configuration with the best hyperparameters
trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config=train_loop_config,
    scaling_config=ScalingConfig(num_workers=num_workers, use_gpu=use_gpu),
    datasets={
        "train": ray_datasets["train"],
        "eval": ray_datasets["validation"],
    },
    run_config=RunConfig(
        checkpoint_config=CheckpointConfig(
            num_to_keep=1,
            checkpoint_score_attribute="eval_loss",
            checkpoint_score_order="min",
        ),
    ),
)

# Start the training process with the best hyperparameters
result = trainer.fit()

2025-07-14 05:40:10,597	WARNING callback.py:136 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`



View detailed results here: /root/ray_results/TorchTrainer_2025-07-14_05-40-10


(TrainTrainable pid=11035) 2025-07-14 05:40:16.035236: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(TrainTrainable pid=11035) 2025-07-14 05:40:16.052348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(TrainTrainable pid=11035) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(TrainTrainable pid=11035) E0000 00:00:1752471616.073427   11035 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(TrainTrainable pid=11035) E0000 00:00:1752471616.079958   11035 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempt


Training started with configuration:
+------------------------------------------+
| Training config                          |
+------------------------------------------+
| train_loop_config/batch_size          16 |
| train_loop_config/epochs               2 |
| train_loop_config/learning_rate   0.0002 |
| train_loop_config/weight_decay       0.1 |
+------------------------------------------+


(TorchTrainer pid=11035) Started distributed worker processes: 
(TorchTrainer pid=11035) - (node_id=dfac881cdf0b95ec3c65ef7c281e0d5ea5b341dd9a6d97ecd807b1a3, ip=172.28.0.12, pid=11136) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=11136) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=11136) 2025-07-14 05:40:26.043580: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(RayTrainWorker pid=11136) 2025-07-14 05:40:26.059935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(RayTrainWorker pid=11136) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(RayTrainWorker 

...

(SplitCoordinator pid=11256) Registered dataset logger for dataset eval_36_2
(SplitCoordinator pid=11256) Starting execution of Dataset eval_36_2. Full logs are in /tmp/ray/session_2025-07-14_05-29-01_733730_2160/logs/ray-data
(SplitCoordinator pid=11256) Execution plan of Dataset eval_36_2: InputDataBuffer[Input] -> OutputSplitter[split(1, equal=True)]
(SplitCoordinator pid=11256) ✔️  Dataset eval_36_2 execution finished in 0.11 seconds
(RayTrainWorker pid=11136) /tmp/ipython-input-15-1470896377.py:78: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
(RayTrainWorker pid=11136) `get_trial_dir` is deprecated because the concept of a `Trial` will soon be removed in Ray Train.Ray Train will no longer assume that it's running within a Ray Tune `Trial` in the future. See this issue for more context and migration options: https://github.com/ray-project/r

(RayTrainWorker pid=11136) Final evaluation: {'eval_loss': 0.8720429539680481, 'eval_accuracy': 0.6607142857142857, 'eval_f1': 0.4612159329140461, 'eval_runtime': 0.2018, 'eval_samples_per_second': 277.539, 'eval_steps_per_second': 19.824, 'epoch': 1.4666666666666668}


(RayTrainWorker pid=11136) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/TorchTrainer_2025-07-14_05-40-10/TorchTrainer_01ca6_00000_0_2025-07-14_05-40-10/checkpoint_000002)
2025-07-14 05:40:49,006	WARNING experiment_state.py:206 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds and may become a bottleneck. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this warning by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0). Set it to 0 to completely suppress this warning.



Training finished iteration 3 at 2025-07-14 05:40:48. Total running time: 38s
+-----------------------------------------+
| Training result                         |
+-----------------------------------------+
| checkpoint_dir_name   checkpoint_000002 |
| time_this_iter_s                1.15634 |
| time_total_s                   26.65713 |
| training_iteration                    3 |
| eval_accuracy                   0.66071 |
| eval_loss                       0.87204 |
+-----------------------------------------+
Training saved a checkpoint for iteration 3 at: (local)/root/ray_results/TorchTrainer_2025-07-14_05-40-10/TorchTrainer_01ca6_00000_0_2025-07-14_05-40-10/checkpoint_000002

Training completed after 3 iterations at 2025-07-14 05:40:50. Total running time: 39s


2025-07-14 05:40:50,358	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/TorchTrainer_2025-07-14_05-40-10' in 0.3288s.


## Save model from checkpoint

In [ ]:
wandb.finish()

In [ ]:
result.checkpoint

Checkpoint(filesystem=local, path=/root/ray_results/TorchTrainer_2025-07-14_05-40-10/TorchTrainer_01ca6_00000_0_2025-07-14_05-40-10/checkpoint_000002)

In [ ]:
with result.checkpoint.as_directory() as checkpoint_dir:
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
    print(f"Model loaded from {checkpoint_dir}")


Model loaded from /root/ray_results/TorchTrainer_2025-07-14_05-40-10/TorchTrainer_01ca6_00000_0_2025-07-14_05-40-10/checkpoint_000002



## Share the model

To share the model via Hugging Face, additional steps are necessary.

You have get your authentication token from the Hugging Face website. If you're not registered yet, create an account [here](https://huggingface.co/join). Subsequently, run the cell below and enter your username and password:

In [ ]:
#notebook_login()
#model.push_to_hub()